# Streamfunction Analysis

Here is the graphical rendering of the streamfunctions.py script. It contains the 
mean meridional streamfunction and its tendency by month, and the annual average 
streamfunction and its tendency over the period 1995&mdash;2024. It is based on the 
monthly averaged meridional wind field on pressure levels as generated by ERA5. 

Imports and defaults. 

In [ ]:
import os
from datetime import datetime
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt 
from matplotlib.ticker import MultipleLocator

axeslinewidth = 0.5 
plt.rcParams.update( {
  'font.family': "Times New Roman", 
  'font.size': 10,  
  'font.weight': "normal", 
  'text.usetex': True, 
  'xtick.major.width': axeslinewidth, 
  'xtick.minor.width': axeslinewidth, 
  'ytick.major.width': axeslinewidth, 
  'ytick.minor.width': axeslinewidth, 
  'axes.linewidth': axeslinewidth } ) 

inputfile = "era5_streamfunction_trend.nc"

sfunits = 1.0e9       # 10**9 kg/s = 1 Mton/s

### Annual average streamfunction and its trend


In [ ]:
outputfile = "era5_streamfunction_trend.pdf"

d = Dataset( inputfile, 'r' )
lats = d.variables['latitude'][:]
levels = d.variables['level'][:]
sf = d.variables['annual_streamfunction_mean'][:]
sft = d.variables['annual_streamfunction_trend'][:]
u = d.variables['annual_surface_uwnd_mean'][:]
d.close()

#  Define streamfunction levels. 

sf_major_levels = np.arange( -200, 201, 50, dtype=np.int32 ).tolist()
sf_minor_levels = np.arange( -200, 201, 10, dtype=np.int32 ).tolist()

print( "Major levels at 50 Mtons/s intervals; minor levels at 10 Mtons/s intervals" )

#  Eliminate major levels from minor levels array. 

x = sf_minor_levels
sf_minor_levels = [ a for a in sf_minor_levels if a not in sf_major_levels ]

sf_minor_levels = np.array( sf_minor_levels )
sf_major_levels = np.array( sf_major_levels )

#  Initialize figure, axes. 

fig = plt.figure( figsize=(6,4.2) )
pos = [ 0.11, 0.40, 0.77, 0.58 ]
ax = fig.add_axes( pos )

#  Define x coordinate. 

xticks = np.arange( -90, 90.1, 30 )
xtickv = []
for xtick in xticks: 
    if xtick > 0: 
        xtickv.append( r'{:}$^\circ$N'.format( int( abs( xtick ) ) ) )
    elif xtick == 0: 
        xtickv.append( 'Eq' )
    else: 
        xtickv.append( r'{:}$^\circ$S'.format( int( abs( xtick ) ) ) )

ax.set_xlim( -90, 90 )
ax.set_xticks( xticks )
# ax.set_xticklabels( xtickv )
ax.set_xticklabels( [] )
ax.xaxis.set_minor_locator( MultipleLocator(10) )
ax.invert_xaxis()
# ax.set_xlabel( "Latitude" )

#  Define y coordinate. 

ax.set_ylim( 0.0, 1000.0 )
ax.set_yticks( np.arange( 200, 1000.1, 200 ) )
ax.yaxis.set_minor_locator( MultipleLocator(100) )
ax.invert_yaxis()
ax.set_ylabel( "Pressure [hPa]" )

ii = ( sf_major_levels > 0.0 )
ax.contour( lats, levels, sf/sfunits, levels=sf_major_levels[ii], colors="k", linewidths=1.0 )
ii = ( sf_major_levels < 0.0 )
ax.contour( lats, levels, sf/sfunits, levels=sf_major_levels[ii], colors="k", linewidths=1.0, linestyles="dashed" )
ii = ( sf_major_levels == 0.0 )
ax.contour( lats, levels, sf/sfunits, levels=sf_major_levels[ii], colors="#FF00FF", linewidths=1.0 )
ii = ( sf_minor_levels > 0.0 )
ax.contour( lats, levels, sf/sfunits, levels=sf_minor_levels[ii], colors="k", linewidths=0.5 )
ii = ( sf_minor_levels < 0.0 )
ax.contour( lats, levels, sf/sfunits, levels=sf_minor_levels[ii], colors="k", linewidths=0.5, linestyles="dashed" )

ax.text( 85, 120, "(a)" )

last_ax = ax.contourf( lats, levels, sft/sfunits*10, levels=np.arange(-5,5.1,0.2), cmap="bwr", extend="both" )

cax = fig.add_axes( [0.90,pos[1],0.015,pos[3]] )
plt.colorbar( last_ax, cax, orientation="vertical", extend="both", 
             ticks=np.arange(-4,5,2), label="Tendency [Mtons/s/dec]" )

#  Zonal wind plot. 

npos = [ pos[0], 0.09, pos[2], 0.26 ]
ax = fig.add_axes( npos )

ax.set_xlim( -90, 90 )
ax.set_xticks( xticks )
ax.set_xticklabels( xtickv )
ax.xaxis.set_minor_locator( MultipleLocator(10) )
ax.invert_xaxis()

yticks = np.arange( -8, 8.01, 4 )
ax.set_ylim( -8, 8 )
ax.set_yticks( yticks )
ax.yaxis.set_minor_locator( MultipleLocator(1) )
ax.set_ylabel( r"Wind $\bar{u}$ [m/s]" )

ax.plot( lats, u, linewidth=0.5, color="#000000" )
ax.fill_between( lats, u, where=(u>=0.0), color="#0040FF", interpolate=True )
ax.fill_between( lats, u, where=(u<=0.0), color="#FF4000", interpolate=True )

#  Grid. 

for y in yticks: 
    ax.plot( [-90,90], [y,y], linewidth=0.5, linestyle="--", color="#808080" )
for x in xticks: 
    ax.plot( [x,x], [yticks[0],yticks[-1]], linewidth=0.5, linestyle="--", color="#808080" )

ax.text( 85, 5.0, "(b)" )

ii = np.logical_and( lats[:-1] >= 25, lats[:-1] <= 50.0 )
ii = np.logical_and( ii, u[:-1] * u[1:] <= 0.0 )
i = np.argwhere( ii ).flatten()[0]
t = -u[i] / ( u[i+1] - u[i] )
xn = lats[i] * (1-t) + lats[i+1] * t

ii = np.logical_and( lats[:-1] >= -50, lats[:-1] <= -25.0 )
ii = np.logical_and( ii, u[:-1] * u[1:] <= 0.0 )
i = np.argwhere( ii ).flatten()[0]
t = -u[i] / ( u[i+1] - u[i] )
xs = lats[i] * (1-t) + lats[i+1] * t

kwargs = dict( clip_on=False, lw=4, head_width=2, head_length=1.0, color="#303030" )
ax.arrow( xn, 11.0, 0.0, -8.0, **kwargs )
ax.arrow( xs, 11.0, 0.0, -8.0, **kwargs )

print( f'Saving to {outputfile}' )
fig.savefig( outputfile )


### ERA5 streamfunction by month

In [ ]:
outputfile = "era5_streamfunction_trend_by_month.pdf"

d = Dataset( inputfile, 'r' )

lats = d.variables['latitude'][:]
levels = d.variables['level'][:]

#  Define streamfunction levels. 

sf_major_levels = np.arange( -200, 201, 50, dtype=np.int32 ).tolist()
sf_minor_levels = np.arange( -200, 201, 10, dtype=np.int32 ).tolist()

print( "Major levels at 50 Mtons/s intervals; minor levels at 10 Mtons/s intervals" )

#  Eliminate major levels from minor levels array. 

x = sf_minor_levels
sf_minor_levels = [ a for a in sf_minor_levels if a not in sf_major_levels ]

sf_minor_levels = np.array( sf_minor_levels )
sf_major_levels = np.array( sf_major_levels )

#  Initialize figure, axes. 

fig = plt.figure( figsize=(9,5) )

#  Define x coordinate. 

xticks = np.arange( -90, 90.1, 30 )
xtickv = []
for xtick in xticks: 
    if xtick > 0: 
        xtickv.append( '{:}$^\circ$N'.format( int( abs( xtick ) ) ) )
    elif xtick == 0: 
        xtickv.append( 'Eq' )
    else: 
        xtickv.append( '{:}$^\circ$S'.format( int( abs( xtick ) ) ) )

#  Loop over month

nx, ny = 4, 3

for imonth in range(12): 

    sf = d.variables['streamfunction_mean'][imonth,:,:]
    sft = d.variables['streamfunction_trend'][imonth,:,:]

    ix, iy = ( imonth % nx ), ny - int(imonth/nx) - 1

    pos = [ 0.06 + 0.22*ix, 0.09 + 0.30*iy, 0.19, 0.26 ]
    ax = fig.add_axes( pos )

    ax.set_xlim( -90, 90 )
    ax.set_xticks( xticks )
    ax.xaxis.set_minor_locator( MultipleLocator(10) )
    ax.invert_xaxis()

    ax.set_ylim( 0.0, 1000.0 )
    ax.set_yticks( np.arange( 200, 1000.1, 200 ) )
    ax.yaxis.set_minor_locator( MultipleLocator(100) )
    ax.invert_yaxis()

    if ix == 0: 
        ax.set_ylabel( "Pressure [hPa]" )
    else: 
        ax.set_yticklabels( [] )
        
    if iy == 0: 
        ax.set_xticklabels( xtickv, rotation=-60 )
        # ax.set_xlabel( "Latitude" )
    else: 
        ax.set_xticklabels( [] )

    label = '({:}) {:}'.format( chr(ord("a")+imonth), datetime(2000,imonth+1,1).strftime("%b") )
    ax.text( 85, -25.0, label )
    
    ii = ( sf_major_levels >= 0.0 )
    ax.contour( lats, levels, sf/sfunits, levels=sf_major_levels[ii], colors="k", linewidths=0.7 )
    ii = ( sf_major_levels < 0.0 )
    ax.contour( lats, levels, sf/sfunits, levels=sf_major_levels[ii], colors="k", linewidths=0.7, linestyles="dashed" )

    ii = ( sf_minor_levels >= 0.0 )
    ax.contour( lats, levels, sf/sfunits, levels=sf_minor_levels[ii], colors="k", linewidths=0.3 )
    ii = ( sf_minor_levels < 0.0 )
    ax.contour( lats, levels, sf/sfunits, levels=sf_minor_levels[ii], colors="k", linewidths=0.3, linestyles="dashed" )

    last_ax = ax.contourf( lats, levels, sft/sfunits*10, levels=np.arange(-5,5.1,0.2), cmap="bwr", extend="both" )

pos = [ 0.93, 0.11, 0.01, 0.82 ]
cax = fig.add_axes( pos )
plt.colorbar( last_ax, cax, orientation="vertical", extend="both", 
             ticks=np.arange(-4,5,2), label="Tendency [Mtons/s/dec]" )

d.close()

print( f'Saving to {outputfile}' )
fig.savefig( outputfile )
